In [77]:
SEASON = '2021-22'

In [78]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES_2020_21
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

p_df.head(50)

,Player,PlayerID,Tms,Pos,Age,G,GS,MP,FG,FGA,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins,PlayerScore
449,Nikola Jokić,jokicni01,DEN,C,26,74,74,33.5,10.3,17.7,...,10.8,4.5,15.2,0.296,9.2,4.5,13.7,9.8,48.0,22.060036
206,Giannis Antetokounmpo,antetgi01,MIL,PF,27,67,67,32.9,10.3,18.6,...,9.2,3.7,12.9,0.281,7.6,3.5,11.2,7.4,51.0,18.531342
295,Joel Embiid,embiijo01,PHI,C,27,68,68,33.8,9.8,19.6,...,7.9,4.1,12.0,0.252,7.2,2.0,9.2,6.5,51.0,16.665660
383,Luka Dončić,doncilu01,DAL,PG,22,65,65,35.4,9.9,21.6,...,3.8,3.8,7.6,0.159,6.4,1.8,8.2,5.9,52.0,14.048283
346,Kevin Durant,duranke01,BRK,PF,33,55,55,37.2,10.5,20.3,...,6.4,2.0,8.4,0.198,6.4,0.7,7.2,4.8,44.0,12.051955
279,Jayson Tatum,tatumja01,BOS,SF,23,76,76,35.9,9.3,20.6,...,5.0,4.6,9.6,0.169,4.3,0.6,4.9,4.8,51.0,11.556941
553,Trae Young,youngtr01,ATL,PG,23,76,76,34.9,9.4,20.3,...,9.0,1.0,10.0,0.181,7.1,-2.0,5.2,4.8,43.0,11.402080
374,LeBron James,jamesle01,LAL,SF,37,56,56,37.2,11.4,21.8,...,5.2,2.3,7.5,0.172,6.9,0.8,7.7,5.1,33.0,11.358011
331,Karl-Anthony Towns,townska01,MIN,C,26,74,74,33.5,8.7,16.4,...,7.1,3.2,10.3,0.199,4.5,0.5,5.0,4.4,46.0,10.990553
87,Chris Paul,paulch01,PHO,PG,36,65,65,32.9,5.6,11.3,...,5.8,3.6,9.4,0.210,3.1,2.3,5.4,4.0,64.0,10.845223
